In [1]:
import ROOT

Welcome to JupyROOT 6.28/04


In [2]:
ROOT.gSystem.Load("libFCCAnalyses")
ROOT.gSystem.Load("libedm4hep")

1

In [3]:
# FIXME: find a better way to do this
# ROOT.gSystem.Load("/cvmfs/sw-nightlies.hsf.org/key4hep/releases/2023-05-30/x86_64-almalinux9-gcc11.3.1-opt/marlinutil/4bba4d10fc1c448213e83251f689370fc2d43c9e=develop-zhkc6e/lib/libMarlinUtil.so")

In [4]:
# needed to be able to include marlinutil/HelixClass
# ROOT.gROOT.ProcessLine(".include /cvmfs/sw-nightlies.hsf.org/key4hep/releases/2023-05-30/x86_64-almalinux9-gcc11.3.1-opt/ced/56f3bc90862e7bd4fa5657e638cceea50b368ed7=develop-lhxqn2/include")

In [5]:
# ROOT.gSystem.GetIncludePath()

In [6]:
ROOT.gInterpreter.Declare("#include <marlinutil/HelixClass_double.h>")
ROOT.gInterpreter.Declare("#include <marlinutil/HelixClass.h>")

True

In [7]:
# I hate this :(
_foo = ROOT.dummyLoader
_bar = ROOT.edm4hep.TrackState()

In [8]:
%jsroot on

In [9]:
%%cpp
ROOT::VecOps::RVec<int> MCTruthTrackIndex(ROOT::VecOps::RVec<int> trackIndex,
                                          ROOT::VecOps::RVec<int> mcIndex,
                                          ROOT::VecOps::RVec<edm4hep::MCParticleData> mc)
{
    ROOT::VecOps::RVec<int> res;
    res.resize(mc.size(), -1);

    for (size_t i = 0; i < trackIndex.size(); i++) {
        res[mcIndex[i]] = trackIndex[i];
    }
    return res;
}

In [10]:
df = ROOT.RDataFrame("events","reco_output/single_mu-_10GeV_89deg_lcio_rec_edm4hep.root")
#df = ROOT.RDataFrame("events","reco_output/single_e-_10GeV_90deg_lcio_rec_2023-06-22_edm4hep.root")
#df = ROOT.RDataFrame("events","reco_output/single_mu-_10GeV_90deg_lcio_rec_newconv_edm4hep.root")
#df = ROOT.RDataFrame("events","reco_output/single_mu-_100GeV_89deg_lcio_rec_newconv_edm4hep.root")
#df = ROOT.RDataFrame("events","reco_output/single_e-_10GeV_90deg_lcio_rec_newconv_edm4hep.root")
#df = ROOT.RDataFrame("events","Output/REC/REC_mu_10deg_10GeV_1000evt_edm4hep.root")

In [11]:
df2 = (df
        #.Alias("MCTrackAssociations0", "MCTruthSiTracksLink#0.index")
        #.Alias("MCTrackAssociations1", "MCTruthSiTracksLink#1.index")
        .Alias("MCTrackAssociations0", "_MCTruthSiTracksLink_rec.index")
        .Alias("MCTrackAssociations1", "_MCTruthSiTracksLink_sim.index")
        .Define("GunParticle_index", "MCParticles.generatorStatus == 1")
        .Define("GunParticle", "MCParticles[GunParticle_index][0]")
        .Define("GunParticle_lv", "ROOT::Math::PxPyPzMVector(GunParticle.momentum.x, GunParticle.momentum.y, GunParticle.momentum.z, GunParticle.mass)")
        .Define("GunParticle_e", "GunParticle_lv.E()")
        .Define("GunParticle_p", "GunParticle_lv.P()")
        .Define("GunParticle_pt", "GunParticle_lv.Pt()")
        .Define("num_acc_residual", "10 - GunParticle_p")
        #.Define("trackStates_IP", "SiTracks_Refitted_1[SiTracks_Refitted_1.location == edm4hep::TrackState::AtIP]")
        #.Define("trackStates_IP", "SiTracks_Refitted_1[SiTracks_Refitted_1.location == 1]")
        .Define("trackStates_IP", "_SiTracks_Refitted_trackStates[_SiTracks_Refitted_trackStates.location == 1]")
        .Define("MC2TrackIndex", "MCTruthTrackIndex(MCTrackAssociations0, MCTrackAssociations1, MCParticles)")
        .Define("GunParticleTrackIndex", "MC2TrackIndex[GunParticle_index][0]")
        .Define("GunParticleTSIP", "trackStates_IP[GunParticleTrackIndex]")
        .Define("GunParticleTSIP_d0", "GunParticleTSIP.D0")
        .Define("GunParticleTSIPHelix", "auto h = HelixClass_double(); h.Initialize_Canonical(GunParticleTSIP.phi, GunParticleTSIP.D0, GunParticleTSIP.Z0, GunParticleTSIP.omega, GunParticleTSIP.tanLambda, 2); return h;")
        .Define("GunParticleTSIP_p", "auto p = GunParticleTSIPHelix.getMomentum(); return sqrt(p[0]*p[0]+p[1]*p[1]+p[2]*p[2]);")
        .Define("GunParticleTSIP_pt", "GunParticleTSIPHelix.getPXY()")
        .Define("GunParticleMCMom", "std::vector<double> v = {GunParticle.momentum.x, GunParticle.momentum.y, GunParticle.momentum.z}; return v;")
        .Define("GunParticleMCPos", "std::vector<double> v = {GunParticle.vertex.x, GunParticle.vertex.y, GunParticle.vertex.z}; return v;")
        .Define("GunParticleMCHelix", "auto h = HelixClass_double(); h.Initialize_VP(GunParticleMCPos.data(), GunParticleMCMom.data(), -1, 2); return h;")
        .Define("GunParticleMCHelix_d0", "GunParticleMCHelix.getD0()")
        .Define("Delta_pt", "GunParticleTSIP_pt - GunParticle_pt")
        .Define("Delta_pt_over_pt2", "Delta_pt / (GunParticle_pt * GunParticle_pt)")
        .Define("pt_inv", "1 / GunParticleTSIP_pt")
)

In [12]:
h_gs2 = df2.Histo1D("MCParticles.generatorStatus")
h_gun_e = df2.Histo1D("GunParticle_e")
h_gun_p = df2.Histo1D("GunParticle_p")
h_num_acc = df2.Histo1D("num_acc_residual")
h_gps = df2.Histo1D("GunParticleTSIP_d0")
h_track_p = df2.Histo1D("GunParticleTSIP_p")
h_gps2 = df2.Histo1D("GunParticleMCHelix_d0")
h_delta_pt = df2.Histo1D("Delta_pt")
h_raw_res_pt = df2.Histo1D("Delta_pt_over_pt2")
h_raw_res_v2_pt = df2.Histo1D("pt_inv")

In [16]:
f = ROOT.TF1("f_test", "ROOT::Math::crystalball_function(x, [0], [1], [2], [3])")
sigma = h_raw_res_pt.GetRMS()
f.SetParameters(sigma, 1, 0.5*sigma, 0)

In [17]:
ROOT.gStyle.SetOptFit(1111)
f_num_acc = h_num_acc.Fit("gaus")
#f_raw_res_pt = h_raw_res_pt.Fit("gaus")
f_raw_res_pt = h_raw_res_pt.Fit(f)
f_raw_res_v2_pt = h_raw_res_v2_pt.Fit("gaus")

 FCN=61.6775 FROM MIGRAD    STATUS=CONVERGED      73 CALLS          74 TOTAL
                     EDM=6.52975e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.55240e+01   1.09851e+00   3.22112e-03   3.49869e-05
   2  Mean         2.08821e-08   7.02342e-09   2.71210e-11  -5.07883e+04
   3  Sigma        1.94913e-07   6.64594e-09   3.47974e-05   5.01930e-03
 FCN=980.84 FROM HESSE     STATUS=FAILED         11 CALLS         268 TOTAL
                     EDM=2.60588e-13    STRATEGY= 1  ERROR MATRIX UNCERTAINTY 100.0 per cent
  EXT PARAMETER                APPROXIMATE        STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           2.14011e+01   4.84055e-04   2.24727e+01   0.00000e+00
   2  p1           4.45576e-01   1.41421e+00   2.35021e-01   0.00000e+00
   3  p2           2.

In [15]:
c_gs2 = ROOT.TCanvas()
h_gs2.Draw()
c_gs2.Draw()

c_gun_e = ROOT.TCanvas()
h_gun_e.Draw()
c_gun_e.Draw()

c_gun_p = ROOT.TCanvas()
h_gun_p.Draw()
c_gun_p.Draw()

c_track_p = ROOT.TCanvas()
h_track_p.Draw()
c_track_p.Draw()

c_delta_pt = ROOT.TCanvas()
h_delta_pt.Draw()
c_delta_pt.Draw()

c_raw_res_pt = ROOT.TCanvas()
h_raw_res_pt.Draw()
c_raw_res_pt.Draw()

c_raw_res_v2_pt = ROOT.TCanvas()
h_raw_res_v2_pt.Draw()
c_raw_res_v2_pt.Draw()

c_num_acc = ROOT.TCanvas()
h_num_acc.Draw()
c_num_acc.Draw()

c_gps = ROOT.TCanvas()
h_gps.Draw()
c_gps.Draw()

c_gps2 = ROOT.TCanvas()
h_gps2.Draw()
c_gps2.Draw()